# **Aplicación del K-means a los datos filtrados**

## *Objetivo:* Aplica el algoritmo K-means para agrupar datos en el número total de clusters hallado anteriormente y utiliza los  parámetros obtenidos por grid_search.best_params. Luego, asigna el número de clusters a cada elemento en el DataFrame original para facilitar su identificación.

### 1. Se cargan librerías y se declaran variables

In [1]:
### Se cargan librerias ###
import json
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore")
file='datos/df_replace.csv'

### Se carga archivo con los grid_search.best_params_ ####
df_replace = pd.read_csv(file)
with open('datos/grid_search.json', 'r') as file:
    grid_search = json.load(file)
X = np.load('datos/X.npy')

### 2. Se declaran los parámetros para el K-menas y se ejecuta el algoritmo

In [2]:
### Se realiza prediccion de etiquetas con los parametros hallados ###
kmeans = KMeans(n_clusters = 7, 
            init=grid_search['init'], 
            max_iter=grid_search['max_iter'], 
            tol=grid_search['tol'],
            algorithm=grid_search['algorithm'], 
            random_state=grid_search['random_state'])
y_kmeans = kmeans.fit_predict(X)

### Se asigna el numero de cluster a cada elemento del DataFrame original ###
df_replace['cluster'] = y_kmeans + 1
df_replace.to_csv('datos/df_cluster.csv', index=False)
df_replace.head()

,Formule,Ele,BG,Structure,CA,cluster
0,BaS3,Ba-S,1.3913,2,62546.518850,3
1,CdF2,Cd-F,2.8977,1,1582.356544,4
2,BeTe,Be-Te,2.0173,1,10109.303511,4
3,UO3,O-U,1.6350,1,228128.774945,1
4,Rb2Te,Rb-Te,1.8766,1,65916.018982,3


### 3. Se agrega la columna *Compuesto* al DataFrame

In [3]:
### Se itera sobre cada elemento de df_replace para contar cuantos elementos componen al material ###
no_eles=[]
df=df_replace
for p in range(len(df)):
    if len(df['Ele'].values[p].split('-'))==2:
        no_eles.append('Binario')
    elif len(df['Ele'].values[p].split('-'))==3:
        no_eles.append('Ternario')
    elif len(df['Ele'].values[p].split('-'))==4:
        no_eles.append('Cuaternario')
    elif len(df['Ele'].values[p].split('-'))==1:
        no_eles.append('Simple')

### Se guardan los resultados en el DadaFrame final ###
df['Compuesto']=no_eles
df_final=df[(df['Compuesto']=='Binario') & (df['cluster']==4) & (df['Structure']==1)].sort_values('BG',ascending=0)
#df_final.to_csv('datos/df_final.csv', index=False) #Descomanetar la línea si es necesario
df_final.head()

,Formule,Ele,BG,Structure,CA,cluster,Compuesto
147,LiH,H-Li,2.9810,1,10195.546816,4,Binario
107,Li2Se,Li-Se,2.9746,1,4373.008538,4,Binario
154,RbH,H-Rb,2.9163,1,6317.384331,4,Binario
1,CdF2,Cd-F,2.8977,1,1582.356544,4,Binario
507,GdF3,F-Gd,2.8622,1,3692.417624,4,Binario
